In [2]:
import pandas as pd
import json
import requests 
import time
import random
import tqdm
import os

In [2]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept':'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,th;q=0.7,zh-CN;q=0.6,zh;q=0.5',
    'X-Guest-Token': 'Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2',
    'Connection' : 'keep-alive',
    'TE': 'Trailers'
}

params = {
    'limit':'5',
    'include': 'comments,contribute_info,attribute_vote_summary',
}

In [21]:
def product_parser(json):
    pInfo = {}

    pInfo['id'] = json.get('id')
    pInfo['sku'] = json.get('sku')
    pInfo['name'] = json.get('name')
    pInfo['discription'] = json.get('short_description')
    pInfo['original_price'] = json.get('original_price')
    pInfo['list_price'] = json.get('list_price')
    pInfo['price'] = json.get('price')
    pInfo['alltime_quantity_sold'] = json.get('all_time_quantity_sold',{})
    pInfo['quantity_sold'] = json.get('quantity_sold', {}).get('value')
    pInfo['inventory_status'] = json.get('inventory_status')
    pInfo['fulfillment_type'] = json.get('inventory', {}).get('fulfillment_type')
    pInfo['brand'] = json.get('brand', {}).get('name')
    pInfo['category_id'] = json.get('categories', {}).get('id')
    pInfo['category'] = json.get('categories').get('name')
    pInfo['review_count'] = json.get('review_count')
    pInfo['rating_average'] = json.get('rating_average')
    pInfo['favourite_count'] = json.get('favourite_count')
    pInfo['ASA_cashback'] = json.get('asa_cashback_widget', {}).get('text')
    pInfo['pay_later'] = json.get('installment_info_v2',{})
    pInfo['current_seller'] = json.get('current_seller', {}).get('name')
    pInfo['date_created'] = json.get('day_ago_created')
    pInfo['video_url'] = json.get('video_url',{})

    return pInfo


In [3]:
folder = 'D:\TIKI\GIAY-NU(done)'
category_list = []
for file_name in os.listdir(folder):
    file_path = os.path.join(folder,file_name)
    id_list = pd.read_csv(file_path,usecols=['product_id','category_name'])
    category_list.append(id_list)

category_list_df = pd.concat(category_list)
# category_list_df.to_csv('giay-nu-id.csv')

In [5]:
category_list_df = category_list_df.drop_duplicates()

In [6]:
category_list_final = category_list_df['product_id'].to_list()
print(len(category_list_final))

24926


In [8]:
category_product_info = []
error_list = []

for id in (category_list_final):
    try:
        response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=params)
        if response.status_code == 200:
            print('crawling product id {}'.format(id))
            category_product_info.append(product_parser(response.json()))
        else:
            print('cannot crawl product id {}!'.format(id))
            error_list.append(id)
    except Exception as e:
        print('An error occurred while crawling product id {}: {}'.format(id, e))
        error_list.append(id)
        continue


crawling product id 252071637
An error occurred while crawling product id 252071637: Expecting value: line 1 column 1 (char 0)
crawling product id 208439152
An error occurred while crawling product id 208439152: Expecting value: line 1 column 1 (char 0)
crawling product id 105883794
An error occurred while crawling product id 105883794: Expecting value: line 1 column 1 (char 0)
crawling product id 252864898
An error occurred while crawling product id 252864898: Expecting value: line 1 column 1 (char 0)
crawling product id 252865522
An error occurred while crawling product id 252865522: Expecting value: line 1 column 1 (char 0)
crawling product id 146591309
An error occurred while crawling product id 146591309: Expecting value: line 1 column 1 (char 0)
crawling product id 186310083
An error occurred while crawling product id 186310083: Expecting value: line 1 column 1 (char 0)
crawling product id 105711332
An error occurred while crawling product id 105711332: Expecting value: line 1 co

crawling parallel at the same time make the server ignore requests and have lots of error pages

In [16]:
giay_nu_df = pd.DataFrame(category_product_info)
giay_nu_df.to_csv('giay-nu.csv')

In [24]:
while error_list:
    error_list_remained = []
    
    for id in error_list:
        try:
            response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=params)
            if response.status_code == 200:
                print('crawling product id {}'.format(id))
                category_product_info.append(product_parser(response.json()))
            else:
                print('cannot crawl product id {}!'.format(id))
                error_list_remained.append(id)
        except Exception as e:
            print('An error occurred while crawling product id {}: {}'.format(id, e))
            error_list_remained.append(id)
            continue
    if len(error_list_remained) == len(error_list):  #if cannot crawl any better end loop!
        break      
    else:  
        error_list = error_list_remained 
        time.sleep(1)


crawling product id 168195597
An error occurred while crawling product id 168195597: 'NoneType' object has no attribute 'get'
crawling product id 252727208
An error occurred while crawling product id 252727208: 'NoneType' object has no attribute 'get'
crawling product id 261951792
An error occurred while crawling product id 261951792: 'NoneType' object has no attribute 'get'
crawling product id 263030218
An error occurred while crawling product id 263030218: 'NoneType' object has no attribute 'get'
crawling product id 173057514
An error occurred while crawling product id 173057514: 'NoneType' object has no attribute 'get'
crawling product id 177319264
An error occurred while crawling product id 177319264: 'NoneType' object has no attribute 'get'
crawling product id 177618544
An error occurred while crawling product id 177618544: 'NoneType' object has no attribute 'get'
crawling product id 176296906
An error occurred while crawling product id 176296906: 'NoneType' object has no attribut

In [25]:
error_list

[168195597,
 252727208,
 261951792,
 263030218,
 173057514,
 177319264,
 177618544,
 176296906,
 188140105,
 235897605,
 263044406,
 6278319,
 188140105,
 184451033,
 215988228,
 235896991,
 263022879,
 208214972,
 181918305,
 205920890]

In [28]:
#CRAWL INFO FROM MAX IMAGE OF PRODUCT DISPLAY

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept':'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,th;q=0.7,zh-CN;q=0.6,zh;q=0.5',
    'X-Guest-Token': 'Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2',
    'Connection' : 'keep-alive',
    'TE': 'Trailers'
}

params = {
    'limit':'5',
    'include': 'comments,contribute_info,attribute_vote_summary',
}

def product_parser(json):
    pInfo = {}
    pInfo['id'] = json.get('id')
    pInfo['sku'] = json.get('sku')

    images = json.get('images')
    if images:
        max_key = len(images)
        pInfo['max_image_key'] = max_key
    
    return pInfo

id_images_info = []
error_list = []

for id in (category_list_final):
    try:
        response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=params)
        if response.status_code == 200:
            print('crawling product id {}'.format(id))
            id_images_info.append(product_parser(response.json()))
        else:
            print('cannot crawl product id {}!'.format(id))
            error_list.append(id)
    except Exception as e:
        print('An error occurred while crawling product id {}: {}'.format(id, e))
        error_list.append(id)
        continue


crawling product id 252071637
crawling product id 208439152
crawling product id 105883794
crawling product id 252864898
crawling product id 252865522
crawling product id 146591309
crawling product id 186310083
crawling product id 105711332
crawling product id 128269189
crawling product id 124146479
crawling product id 128269542
crawling product id 248089336
crawling product id 100968622
crawling product id 115815396
crawling product id 178537658
crawling product id 124311508
crawling product id 128268591
crawling product id 184760272
crawling product id 124297898
crawling product id 247562600
crawling product id 124777115
crawling product id 85814056
crawling product id 166107627
crawling product id 99698946
crawling product id 173239501
crawling product id 48995670
crawling product id 171856499
crawling product id 48995698
crawling product id 259539350
crawling product id 259539630
crawling product id 242704717
crawling product id 204714674
crawling product id 259538641
crawling produ

In [ ]:
giay_nu_image_df = pd.DataFrame(id_images_info)
giay_nu_df.to_csv('giay-nu-images.csv')

In [ ]:
while error_list:
    error_list_remained = []
    
    for id in error_list:
        try:
            response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=params)
            if response.status_code == 200:
                print('crawling product id {}'.format(id))
                id_images_info.append(product_parser(response.json()))
            else:
                print('cannot crawl product id {}!'.format(id))
                error_list_remained.append(id)
        except Exception as e:
            print('An error occurred while crawling product id {}: {}'.format(id, e))
            error_list_remained.append(id)
            continue
    if len(error_list_remained) == len(error_list):  #if cannot crawl any better end loop!
        break      
    else:  
        error_list = error_list_remained 
        time.sleep(1)
